# 11_data_generate

In this notebook, we will generate dataset including different levels of data with gaussian from 1-4 and noise

In [ ]:
#all_no_test
#def_exp data_generate

### package prepare

In [ ]:
import deerlab as dl
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import skewnorm, vonmises, ttest_rel
from datasets import DatasetDict

### function prepare

In [ ]:
def generate_data_dl( n_duplicate_params: int = 1,
    n_points: int = 192,
    n_dist_pts: int = 128,
    n_gauss_lo : int = 1,
    n_gauss_hi : int = 3,
    dist_mean_lo : float = 15.,
    dist_mean_hi : float = 90.,
    width_lo : float = 0.05,
    width_hi : float = 0.5,
    skew_lo : float = -3.,
    skew_hi : float = 3.,
    rmax_lo : float = 50.,
    rmax_hi : float = 160.,
    tmin_lo = -0.5,
    tmin_hi = 0.,
    tmax_lo : float = 2.,
    tmax_hi : float = 15.,
    dim_lo: float = 2.,
    dim_hi: float = 3.5,
    frac_3d: float = 0.5,
    depth1_lo : float = 0.05,
    depth1_hi : float = 0.6,
    depth2_lo : float = 0.0,
    depth2_hi : float = 0.2,
    depth3_lo : float = 0.0,
    depth3_hi : float = 0.2,
    depth4_lo : float = 0.0,
    depth4_hi : float = 0.01,
    t4_min: float = 0.016,
    t4_max: float = 0.096,
    noise_lo : float = 0.0,
    noise_hi : float = 0.1,
    bckg_lo : float = -6.,
    bckg_hi : float = -0.5,
    vm_kappa: float = 180/np.pi,
    use_pathways: bool = True ):
    '''
    This function can generate simulated signal data based on Pr
    It includes 4 kinds of outputting
    distrs is the pr related with the simulated signal
    decays is the simulated signal and position
    r_list is generated distance
    t_list is the generated time
    
    '''
  
    # decays include the siginal data we simulate
    # distrs is the pr we randomly generate
    # t_list is the time we generate
    # r_list is the distance we generate

    decays = np.zeros( (n_points, 2) )   
    # generate random time (t) and distance(r)
    tmin = np.random.uniform( low=tmin_lo, high=tmin_hi )
    tmax = np.random.uniform( low=tmax_lo, high=tmax_hi ) + tmin
    rmax = np.cbrt( tmax * 5.2e4 )

    maxdist = np.random.uniform( rmax_lo, rmax_hi )
        
    r = np.linspace( 15., maxdist, num=n_dist_pts )
    t = np.linspace( tmin, tmax, num=n_points )
        
    # Check whether use pathways
    if use_pathways:
        pathways = [ [ 1., 0. ] ]
        pathways.extend( [ 
        [ np.random.uniform( depth2_lo, depth2_hi ), tmax ],
        [ np.random.uniform( depth3_lo, depth3_hi ), tmax - np.random.uniform( t4_min, t4_max ) ],
        [ np.random.uniform( depth4_lo, depth4_hi ), tmin ] ]
        )

        K = dl.dipolarkernel( t, r / 10., pathways=pathways )

    else:
        K = dl.dipolarkernel( t, r / 10. )
            
            
        # 
    for _ in range( n_duplicate_params ):
        y = n_gauss_hi

        # set pr
        # the length of amps is the number of guassian in pr
        # the value in amps is the weight of different guassian distribution 
        pr = np.zeros( ( n_dist_pts ) )
        amps = np.random.uniform( size =( np.random.randint( n_gauss_lo, n_gauss_hi + 1 ) ) )
        amps /= amps.sum()
        for amp in amps:
            # add different weight pr into final result
            meandist = np.random.uniform( dist_mean_lo, min( rmax, dist_mean_hi, maxdist ) )
            stdev = np.random.uniform( width_lo, width_hi ) * ( meandist / 2 )
            skew = np.random.uniform( skew_lo, skew_hi )
            pr += amp * skewnorm.pdf( r, skew, meandist, stdev )
        # normalize pr
        pr /= pr.sum()
        distrs = pr
            

        # Set up remaining variables
        depth1 = np.random.uniform( depth1_lo, depth1_hi )
        noise = depth1 * np.random.uniform( noise_lo, noise_hi )
        # generate random backgraound
        bckg = 10 ** np.random.uniform( bckg_lo, bckg_hi )
        dim = 3.
        if np.random.uniform( 0., 1. ) > frac_3d:
            dim = np.random.uniform( dim_lo, dim_hi )

        # set V(t)
        O = K@pr
        O /= O.max()
        V = 1. - depth1 * ( 1. - O)
        V *= np.exp( -1 * ( bckg * np.abs( t ) ) ** ( dim / 3. ) )
        # normalize V
        V /= V.max()
        angle = vonmises.rvs( vm_kappa )

        real = np.cos( angle ) * V + np.random.normal( loc=0, scale=noise, size=( n_points ) )
        imag = np.sin( angle ) * V + np.random.normal( loc=0, scale=noise, size=( n_points ) )

        imag /= real.max()
        real /= real.max()

        decays[:, 0 ] = real
        decays[:, 1 ] = imag
        t_list = t
        r_list = r

    return distrs, decays, t_list, r_list
 
    

In [ ]:
distrs, decays, t_list, r_list = generate_data_dl()

In [ ]:
new_decays = pd.DataFrame({"Signal_real": decays[:,0],"Signal_imag": decays[:,1]})

In [ ]:
new_decays.head()

,Signal_real,Signal_imag
0,0.950551,0.068545
1,0.930748,0.068493
2,0.958633,0.062110
3,0.973773,0.090962
4,0.969109,0.073948
